## Pricing Options Solutions

The Black-Scholes option pricing models for European-style options on
a non-dividend paying stock are::

    c = S0 * N(d1) - K * exp(-r*T)* N(d2)  for a call option and

    p = K*exp(-r*T)*N(-d2) - S0 * N(-d1)   for a put option

where::
     
    d1 = (log(S0/K) + (r + sigma**2 / 2)*T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)

Also:

* `log` is the natural logarithm.
* `N(x)` is the cumulative density function of a standard normal distribution.
* *S0* is the current price of the stock.
* *K* is the strike price of the option.
* *T* is the time to maturity of the option.
* *r* is the (continuously-compounded) risk-free rate of return.
* *sigma* is the stock price volatility.

0. Create a function that returns the call and put options prices
   for using the Black-Scholes formula and the inputs of
   *S0*, *K*, *T*, *r*, and *sigma*.

   Hint:  Use scipy.stats.norm.cdf, and notice that N(x) + N(-x) = 1.


Another approach to pricing options is to simulate instrument price over the
lifetime of the option using Monte Carlo simulation (assuming some model). The
resulting random walk of the stock price can be used to determine the option
pay-off.

The option price is then usually calculated as the average value of the
simulated pay-offs discounted at the risk-free rate of return (exp(-r*T)).

An often-used model for the stock price at time *T* is that it is log-normally
distributed where log(ST) is normally distributed with::

     mean - log(S0) + (mu-sigma**2 / 2) * T 
     
and::
    
     variance - sigma**2 * T

This implies that the *ST* is log-normally distributed with shape parameter
``sigma * sqrt(T)`` and scale parameter ``S0*exp((mu-sigma**2/2)*T)``. 
     
For option pricing, *mu* is the risk-free rate of return and *sigma* is
the volatility.

The value of a call option at maturity is ST-K if ST>K and 0 if ST<K.
The value of a put option at maturity is K-ST if K>ST and 0 if ST>K.

1. Create a function that uses a Monte Carlo method to price vanilla
   call and put options, by drawing samples from a log-normal distribution.

2. Create a function that uses a Monte-Carlo method to price vanilla
   call and put options, by drawing samples from a normal distribution.

3. Compare results from the pricing methods in parts 0, 1, and 2.


In [1]:
# Ensure integer values for prices, etc. will work correctly.
from __future__ import division

from numpy import log, exp, sqrt, maximum
from scipy.stats import lognorm, norm

def bsprices(S0, K, T, r, sigma):
    """Black-Scholes call and put option pricing

    Parameters
    ----------
    S0 :
        Current price of the underlying stock
    K :
        Strike price of the option
    T :
        Time to maturity of the option
    r :
        Risk-free rate of return (continuously-compounded)
    sigma :
        Stock price volatility

    Returns
    -------
    c :
        Call option price
    p :
        Put option price

    Notes
    -----
    r, T, and sigma must be expressed in consistent units of time
    """
    x0 = sigma * sqrt(T)
    erT = exp(-r*T)
    d1 = (log(S0/K) + (r + sigma**2 / 2) * T) / x0
    d2 = d1 - x0
    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)
    c = S0*Nd1 - K*erT*Nd2
    p = K*erT*(1-Nd2) - S0*(1-Nd1)
    return c, p


def mcprices(S0, K, T, r, sigma, N=5000):
    """
    Call and put option prices using log-normal Monte-Carlo method

    Parameters
    ----------
    S0 :
        Current price of the underlying stock
    K :
        Strike price of the option
    T :
        Time to maturity of the option
    r :
        Risk-free rate of return (continuously-compounded)
    sigma :
        Stock price volatility
    N :
        Number of stock prices to simulate

    Returns
    -------
    c :
        Call option price
    p :
        Put option price

    Notes
    -----
    r, T, and sigma must be expressed in consistent units of time        
    """
    scale = S0*exp((r-sigma**2/2)*T)
    shape = sigma*sqrt(T)
    ST_sim = lognorm.rvs(shape,scale=scale, size=N)
    call_pay_off = maximum(ST_sim - K, 0)
    put_pay_off = maximum(K - ST_sim, 0)
    discount = exp(-r*T)
    return (call_pay_off.mean()*discount,
            put_pay_off.mean()*discount)


def mcprices2(S0, K, T, r, sigma, N=5000):
    """
    Call and put option prices using normal Monte-Carlo method

    Parameters
    ----------
    S0 :
        Current price of the underlying stock
    K :
        Strike price of the option
    T :
        Time to maturity of the option
    r :
        Risk-free rate of return (continuously-compounded)
    sigma :
        Stock price volatility

    Returns
    -------
    c :
        Call option price
    p :
        Put option price

    Notes
    -----
    r, T, and sigma must be expressed in consistent units of time
    
    """
    mean = (r - sigma**2/2)*T
    std = sigma*sqrt(T)
    values = norm.rvs(loc=mean, scale=std, size=N)
    ST_sim = S0*exp(values)
    call_pay_off = maximum(ST_sim - K, 0)
    put_pay_off = maximum(K-ST_sim, 0)
    discount = exp(-r*T)
    return (call_pay_off.mean()*discount,
            put_pay_off.mean()*discount)


if __name__ == "__main__":
    S0 = 40 # $40 current price
    K  = 42 # $46 strike price
    T = 3/12.0  # 3 months in units of years
    r = 0.01 # annual risk-free rate of return
    sigma = 0.30 # volatility per annum
    msg = "Call price: %4.2f; Put price: %4.2f"
    c1, p1 = mcprices(S0, K, T, r, sigma, N=10000)    
    print msg % (c1, p1)
    c1, p1 = mcprices2(S0, K, T, r, sigma, N=10000)    
    print msg % (c1, p1)
    c1, p1 = bsprices(S0, K, T, r, sigma)    
    print msg % (c1, p1)


Call price: 1.64; Put price: 3.48
Call price: 1.63; Put price: 3.55
Call price: 1.62; Put price: 3.51
